In [ ]:
import geopandas

## Create data frame from shapefile
Let's make a GeoDataFrame from our postcode dataset. We can do this directly using the `read_file()` geopandas method.  
  
The `read_file()` method references the Fiona library's import functions, and can read from any OGR vector source.

In [ ]:
aus_poas = geopandas.read_file('aus_poas.shp')

This is what our dataframe looks like - it's a table, but with a fully typed spatial geometry column.  
  
The pandas `head()` method returns the first 5 rows with our attributes - POA_NAME, code, state and geometry

In [ ]:
aus_poas.head()

## Plot our data  
Basic plotting is fully implemented on our dataframe via MatPlotLib, and we can get a quick and dirty preview of our data using the `plot()` method on the dataframe

In [ ]:
ax = aus_poas.plot()

## Pandas Methods  
All the common methods for a Pandas dataframe are supported on our GeoPandas dataframe.  
  
We can use the `query()` method to select features. For example, this `query()` returns the Melbourne postcode

In [ ]:
melb = aus_poas.query('code == 3000')
ax = melb.plot()
melb.head()

Let's select all the Victorian postcodes

In [ ]:
vic_poas = aus_poas.query('code >= 3000 & code <= 3999')
ax = vic_poas.plot()
vic_poas.head()

## Performing Dataframe Operations
There are a bunch of useful methods available right on our GeoDataframe, such as `dissolve()`.  
  
Let's create a shape of Victoria using `dissolve()`, based on the Victoria postcode polygons.

In [ ]:
vic_shape = vic_poas.dissolve(by='state')
ax = vic_shape.plot()
vic_shape.head()

> note that this has made the 'state' column the index column, on the far left.

## Projecting & Buffering
Now that we have our Victoria polygon, we want to buffer around it to get adjoining postcodes from the full set.  
  
The vic_shape layer is in GDA94, so let's re-project it to GA Lambert EPSG:3112 so we can calculate in metres. We can do this super quickly and easily using the `to_crs()` method.

In [ ]:
vic_shape_lamb = vic_shape.to_crs(epsg=3112)
ax = vic_shape_lamb.plot()

Now we can buffer in metres, using the `buffer()` method.  
  
The `buffer()` method returns a GeoSeries (a single feature geometry), but we want to keep using our data in a GeoDataFrame, so we need to create a new data frame and then add the resulting buffer GeoSeries.  
  
We create a new empty GeoDataFrame using `geopandas.GeoDataFrame()` with the CRS we defined earlier on the vic_shape object (which we can reference directly with the `.crs` property)

In [ ]:
vic_buffer = geopandas.GeoDataFrame(crs=vic_shape_lamb.crs)

Then we can perform the 10km buffer operation with `buffer(10000)`, and store it in the 'geometry' column on the GeoDataFrame we just created.

In [ ]:
vic_buffer['geometry'] = vic_shape_lamb.buffer(10000)
ax = vic_buffer.plot()

Let's re-project our buffer to get it back to the original CRS, ready to for joining with our full postcode dataset

In [ ]:
vic_buffer_gda = vic_buffer.to_crs(aus_poas.crs)
ax = vic_buffer_gda.plot()

## Do a Spatial Join to Select All Postcodes Inside Victoria and Within 10kms
We can use the 'spatial join' or `sjoin()` method (accessible via the geopandas class) to do an intersect analysis, then query out the features that satisfy the intersect relationship.  
  
Performing `sjoin()` with the `how` parameter set to 'left' keeps all rows from the full set of postcodes, but we can filter them down to just those that intersect by querying the 'index_right' column, which is added as part of the join with the index of the joined feature (in this case our Vic buffer has an index_right id of 'VIC', as noted above)

In [ ]:
vic_plus = geopandas.sjoin(aus_poas, vic_buffer_gda, how='left', op='intersects').query('index_right == "VIC"')
ax = vic_plus.plot()
vic_plus.head()

The result is all of the postcodes in Victoria, and the postcodes in the other states which come within 10kms of the border

## Exporting Our Data
Finally, let's save our data for use in part two of the processing. We can use the `to_file()` method directly from the GeoDataFrame to save in our workspace.

In [ ]:
vic_plus.to_file('./outputs/vic_poas_plus.shp', driver='ESRI Shapefile')

Let's quickly check our output file...

In [ ]:
ax = geopandas.read_file('./outputs/vic_poas_plus.shp').plot()

> I'm also going to save out our dissolved Victoria shape for later

In [ ]:
vic_shape.to_file('./outputs/vic_shape.shp', driver='ESRI Shapefile')

This ends part 1, in part two let's create some random points, join the postcode attributes to them, and create some minimum bounding boxes.